In [ ]:
# autorender_ai_colab.ipynb
# Google Colab notebook to run AutoRender AI backend with ngrok and test requests

# ================================
# 1. Clone the repo
# ================================
!git clone https://github.com/BalajiAIDev/autorender-ai.git
%cd autorender-ai

# ================================
# 2. Install dependencies
# ================================
!pip install -r requirements.txt
!pip install flask-ngrok requests pillow

# ================================
# 3. Setup ngrok (for public URL)
# ================================
!pip install pyngrok
from pyngrok import ngrok

# Kill previous tunnels if any
ngrok.kill()

# Create tunnel for Flask app
public_url = ngrok.connect(5000)
print(f"🚀 Public URL: {public_url}")

# ================================
# 4. Run Flask app
# ================================
import subprocess
import threading

def run_flask():
    subprocess.run(["python", "app.py"])

thread = threading.Thread(target=run_flask)
thread.start()

# ================================
# 5. Test background removal API
# ================================
import time
time.sleep(10)  # Wait for Flask to fully start

import requests
from PIL import Image
from io import BytesIO

# Example public image URL for testing
image_url = "https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png"

print("Sending test request to background removal API...")
response = requests.post(
    f"{public_url}/remove-bg",
    json={"image_url": image_url}
)

if response.status_code == 200:
    result_url = response.json().get("result_url")
    print(f"✅ Background removed image URL: {result_url}")

    # Fetch and display
    img_resp = requests.get(result_url)
    img = Image.open(BytesIO(img_resp.content))
    img.show()
else:
    print("❌ Request failed:", response.status_code, response.text)
